In [4]:
%env BASE_DIR=/home/jupyter/snippets_bkp

env: BASE_DIR=/home/jupyter/snippets_bkp


In [6]:
 !gsutil cp gs://cs327e-open-access/shopify.zip .
 !unzip shopify.zip

Copying gs://cs327e-open-access/shopify.zip...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      
Archive:  shopify.zip
   creating: shopify/
  inflating: __MACOSX/._shopify      
  inflating: shopify/apps_categories.csv  
  inflating: __MACOSX/shopify/._apps_categories.csv  
  inflating: shopify/categories.csv  
  inflating: __MACOSX/shopify/._categories.csv  
  inflating: shopify/reviews.csv     
  inflating: __MACOSX/shopify/._reviews.csv  
  inflating: shopify/.DS_Store       
  inflating: __MACOSX/shopify/._.DS_Store  
  inflating: shopify/key_benefits.csv  
  inflating: __MACOSX/shopify/._key_benefits.csv  
  inflating: shopify/pricing_plan_features.csv  
  inflating: __MACOSX/shopify/._pricing_plan_features.csv  
  inflating: shopify/pricing_plans.csv  
  inflating: __MACOSX/shopify/._pricing_plans.csv  
  inflating: shopify/apps.csv        
  inflating: __MACOSX/shopi

In [1]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()

In [12]:
batch = db.batch()

df_apps_categories = pd.read_csv('shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n').values.tolist()
df_apps = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n').values.tolist()
df_categories = pd.read_csv('shopify/categories.csv', sep=',', header=0, lineterminator='\n').values.tolist()
df_key_benefits = pd.read_csv('shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n').values.tolist()
df_pricing_plan_features = pd.read_csv('shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
column_names = ["pricing_plan_id", "app_id", "feature"]
df_pricing_plan_features = df_pricing_plan_features.reindex(columns=column_names).values.tolist()
df_pricing_plans = pd.read_csv('shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n').values.tolist()
df_reviews = pd.read_csv('shopify/reviews.csv', sep=',', header=0, lineterminator='\n').values.tolist()


In [14]:
#Creating apps with subcollections of key_features,pricing_plans, and reviews

counts = 0
for row_app in df_apps:

    appRecord = {}
    appRecord['id'] = row_app[0] 
    appRecord['url'] = row_app[1]
    appRecord['title'] = row_app[2]
    appRecord['developer'] = row_app[3]
    appRecord['developer_link'] = row_app[4]
    appRecord['icon'] = row_app[5]
    appRecord['rating'] = row_app[6]
    appRecord['reviews_count'] = row_app[7]

    appRef = db.collection('Apps').document(row_app[0])
    batch.set(appRef,appRecord)


    for benefits_row in df_key_benefits: 
        if row_app[0] == benefits_row[0]:
            benefitsRecord = {} 
            benefitsRecord['title'] = benefits_row[1]
            benefitsRecord['description'] = benefits_row[2]
            benefitsRef = appRef.collection('Key_Benefits').document(benefits_row[0])
            batch.set(benefitsRef,benefitsRecord)
            counts += 1
            if counts == 200:
                batch.commit()
                counts = 0
    for pricing_row in df_pricing_plans: 
            if row_app[0] == pricing_row[1]:
                pricingRecord = {} 
                pricingRecord['id'] = pricing_row[0]
                pricingRecord['title'] = pricing_row[2]
                pricingRecord['price'] = pricing_row[3]
                pricingRef = appRef.collection('Pricing_Plans').document(pricing_row[0])
                batch.set(pricingRef,pricingRecord)
                counts += 1
                if counts == 200:
                    batch.commit()
                    counts = 0
    for reviews_row in df_reviews: 
            if row_app[0] == reviews_row[0]:
                reviewRecord = {} 
                reviewRecord['author'] = reviews_row[1]
                reviewRecord['rating'] = reviews_row[2]
                reviewRecord['posted_at'] = reviews_row[3]
                reviewsRef = appRef.collection('Reviews').document(reviews_row[0])
                batch.set(reviewsRef,reviewRecord)
                counts += 1
                if counts == 200:
                    batch.commit()
                    counts = 0

                
        
       


In [45]:
#Creating categories with subcollections of apps_categories and apps 
batch = db.batch()
counts = 0
for row_category in df_categories:

    categoryRecord = {}
    categoryRecord['id'] = row_category[0] 
    categoryRecord['title'] = row_category[1]   
    categoryRef = db.collection('Categories').document(row_category[0])
    batch.set(categoryRef,categoryRecord)


    for app_category_row in df_apps_categories: 
        if row_category[0] == app_category_row[1]:
            for apps in df_apps:
                if(app_category_row[0] == apps[0]):
                    
                    appRecord = {}
                    appRecord['id'] = apps[0]
                    appRecord['url'] = apps[1]
                    appRecord['title'] = apps[2]
                    appRecord['developer'] = apps[3]
                    appRecord['developer_link'] = apps[4]
                    appRecord['icon'] = apps[5]
                    appRecord['rating'] = apps[6]
                    appRecord['reviews_count'] = apps[7]

                    appRef = categoryRef.collection('Apps').document(apps[0])
                    batch.set(appRef,appRecord)
                    counts += 1
                    if counts == 200:
                        batch.commit()
                        counts = 0
 


In [20]:
doc_apps = db.collection('Apps').stream()
count = 0
for i in doc_apps:
    count += 1
    
print("There are {} documents in Apps collection".format(count))

There are 3547 documents in Apps collection


In [65]:
totalNumBenefits = 0
totalNumReviews = 0
totalNumPricingPlans = 0
for app_row in df_apps:
    totalNumBenefits+=len(list(db.collection('Apps').document(app_row[0]).collection('Key_Benefits').stream()))
    totalNumReviews+= len(list(db.collection('Apps').document(app_row[0]).collection('Reviews').stream()))
    totalNumPricingPlans+=len(list(db.collection('Apps').document(app_row[0]).collection('Pricing_Plans').stream()))
    
print("There are {} documents in Apps-Key_Benefits sub-collection".format(totalNumBenefits))
print("There are {} documents in Apps-Reviews sub-collection".format(totalNumReviews))
print("There are {} documents in Apps-Pricing-Plans sub-collection".format(totalNumPricingPlans))

There are 3184 documents in Apps-Key_Benefits sub-collection
There are 1708 documents in Apps-Reviews sub-collection
There are 6275 documents in Apps-Pricing-Plans sub-collection


In [61]:
doc_cats = db.collection('Categories').stream()
count = 0
for i in doc_cats:
    count += 1
    
print("There are {} documents in Categories collection".format(count))

There are 12 documents in Categories collection


In [62]:
totalApps = 0
for app_row in df_categories:
        totalApps+= len(list(db.collection('Categories').document(app_row[0]).collection('Apps').stream()))

    
print("There are {} documents in Categories-Apps sub-collection".format(totalApps))


There are 5200 documents in Categories-Apps sub-collection


In [59]:
query = db.collection('Categories').document(db.collection('Categories').where('title', '==','Productivity').get()[0].to_dict()["id"]).collection('Apps').order_by("rating",direction =firestore.Query.DESCENDING).limit(10).stream()

for results in query:
    print(results.to_dict()["id"],results.to_dict()["title"],results.to_dict()["developer"],results.to_dict()["rating"],results.to_dict()["reviews_count"])


fe5ae45c-379c-42bf-be7d-b5f6e15fc13c Customer Tags Union Works Apps 5.0 21
f99bb1e3-f326-4f10-8901-491652e9809b Order Tagger Union Works Apps 5.0 68
f864e3bd-da0e-41dc-be65-984325331475 SilkRoad ‑ Facebook Auto Ads SilkRoad 5.0 2
f5344e64-9cda-4d97-b198-2aaeb5170518 Xporter Data Export Tool Modd Apps Inc. 5.0 223
ed77a32d-0fa3-458b-b639-e01ea7b78ec0 FraudBlock Fraud Prevention ShopFox 5.0 6
e0e231d6-4988-4a8c-ad84-cf18d0f38738 Automation Fox Automation Fox 5.0 1
d75fa395-3a4d-41c3-88d1-2f802c1e2411 AirPower BaseGenius 5.0 1
d0be03ff-f74d-4fb5-8d2c-79541a1aea1b Excelify Excelify.io 5.0 178
c9a86f8c-817e-4bb5-bbdd-e0e669bd7816 UPC Code Manager Ventures & Adventures 5.0 1
c7578030-19fd-42e1-a5a5-03c7861220aa EzySlips ‑ Shipping Automation Onjection Labs 5.0 219


In [60]:
query = db.collection('Apps').order_by("reviews_count",direction =firestore.Query.DESCENDING).limit(10).stream()

for results in query:
    print(results.to_dict()["id"],results.to_dict()["title"],results.to_dict()["developer"],results.to_dict()["rating"],results.to_dict()["reviews_count"])


d9f142ee-b141-4dc4-9353-173db61d2eb0 Privy ‑ Exit Pop Ups & Email Privy 4.7 23078
78ea0810-c008-4a4e-a82f-de0c790e3286 Free Shipping Bar Hextom 4.9 8737
b88488b0-9912-44d3-b736-224c36f09d95 Sales Pop ‑ Popup Notification CartKit 4.8 6905
e528a60e-94f8-4e92-80e2-5bc6013b8283 BEST Currency Converter Grizzly Apps 4.8 5986
be2640c4-01b5-4d52-9f68-cae8c0734d0d Recart FB Messenger Marketing Recart 4.8 5596
70bff9e0-4316-4cc6-84ce-92fcd1bc6925 EU Cookie Bar ‑ Cookie GDPR Booster Apps 4.7 5259
171816e2-27d4-4552-a65e-ab44a312fe04 Sales Pop Master ‑ Countdown Autoketing 4.8 4931
9025eff0-d714-4df1-930f-43f5582979ad Ultimate Sales Boost Hextom 4.8 4847
404183ac-6d52-4960-85d2-8350f37c2f0b Ads by Varinode Varinode, Inc. 4.9 4245
0380c5ed-7b91-4d4b-9a4e-f6a1c50269dd Quick Announcement Bar Hextom 4.9 4237
